In [1]:
with open('Y.txt', 'r') as f:
    Y = f.read().splitlines()
with open('X.txt', 'r') as f:
    X = f.read().splitlines()

In [11]:
X = np.array(X)
Y = np.array(Y)

In [7]:
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import Dataset
import jax
from jax import numpy as jnp
import optax
from transformers.models.t5.modeling_flax_t5 import FlaxT5ForConditionalGeneration

In [13]:
import pandas as pd

In [8]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
max_length = 512  # 可根据需求调整

In [9]:
def encode(example):
    input_text = example["input_text"]
    target_text = example["target_text"]
    input_encoded = tokenizer(input_text, max_length=max_length, truncation=True, padding="max_length", return_tensors="np")
    target_encoded = tokenizer(target_text, max_length=max_length, truncation=True, padding="max_length", return_tensors="np")
    return {"input_ids": input_encoded["input_ids"], "attention_mask": input_encoded["attention_mask"], "labels": target_encoded["input_ids"]}

In [14]:
data = pd.DataFrame({"input_text": X, "target_text": Y})
dataset = Dataset.from_pandas(data)
dataset = dataset.map(encode, batched=True)

In [15]:
# 将数据集拆分为训练集和验证集
dataset = dataset.shuffle(seed=42)
train_dataset, eval_dataset = dataset.train_test_split(train_size=0.9)

In [16]:
# 模型和优化器
model = FlaxT5ForConditionalGeneration.from_pretrained("t5-small", input_shape=(1, max_length))
optimizer = optax.adamw(learning_rate=1e-4)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [17]:
# 训练循环
def train_step(model, batch, optimizer):
    def loss_fn(params):
        with jax.experimental.maps.mesh([], []):
            logits = model(batch["input_ids"], attention_mask=batch["attention_mask"], params=params).logits
            loss = optax.softmax_cross_entropy(logits, batch["labels"]).mean()
        return loss

    grad_fn = jax.value_and_grad(loss_fn)
    loss, grads = grad_fn(model.params)
    optimizer = optimizer.update(grads)
    return loss, optimizer

In [18]:
epochs = 10
batch_size = 8

In [20]:
for epoch in range(epochs):
    train_dataset = train_dataset.shuffle(seed=42)
    train_loss = 0
    for i in range(0, len(train_dataset), batch_size):
        batch = {k: v[i:i + batch_size] for k, v in train_dataset.items()}
        loss, optimizer = train_step(model, batch, optimizer)
        train_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {train_loss / (len(train_dataset) // batch_size)}")

AttributeError: 'str' object has no attribute 'items'

In [ ]:
# 保存模型
model.save_pretrained("t5-small-trained")